In [1]:
###Importing packages we need###

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
from matplotlib.cm import get_cmap
from matplotlib.colors import LinearSegmentedColormap
import metpy.calc as mpcalc
from metpy.units import units
from matplotlib.lines import Line2D
from numpy import *
import xarray as xr
from netCDF4 import Dataset, num2date
import math
import pygrib
import matplotlib.patches as patches
import cdsapi
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import metpy as mp
import os
import warnings
warnings.filterwarnings("ignore")

print("Done importing modules")

Done importing modules


In [2]:
###Import the 3 CF tables and concatenate###

#Read in the 3 tables#
dfg = pd.read_csv('/projects/zami4053/Synoptic_Research/CSV_DFs/CF_good.csv')
dfmf = pd.read_csv('/projects/zami4053/Synoptic_Research/CSV_DFs/CF_mf_updated.csv')
dfu = pd.read_csv('/projects/zami4053/Synoptic_Research/CSV_DFs/CF_unclear_updated_new.csv')
                  
#Combine into 1 big file with all CF cases and reset index#
dfall = pd.concat([dfg, dfmf, dfu])
dfall.reset_index(drop=True, inplace=True)
#dfall

#Sort the DataFrame by 'date' and del duplicate#
dfall_sort = dfall.sort_values(by='date').reset_index(drop=True)
dfall_sort = dfall_sort.drop(260)
dfall_sort = dfall_sort.sort_values(by='date').reset_index(drop=True)
dfall_sort

#Organize by Low and High Precipitation Events#
df_LP = pd.read_csv('/projects/zami4053/Synoptic_Research/CSV_DFs/CF_Low_Precip.csv')
df_HP = pd.read_csv('/projects/zami4053/Synoptic_Research/CSV_DFs/CF_High_Precip.csv')
df_All = pd.read_csv('/projects/zami4053/Synoptic_Research/CSV_DFs/CF_All_Precip.csv')

df_select = df_All.copy()
df_select

,date,max_value,max_time,file_date,total_precip
0,1950-01-04,12449.629923,1950-01-03 09:00:00,1950-01-04,328.730494
1,1950-01-30,10615.853132,1950-01-29 06:00:00,1950-01-30,87.406158
2,1950-02-22,1695.041284,1950-02-21 16:00:00,1950-02-22,319.826066
3,1950-03-01,2258.049045,1950-02-28 07:00:00,1950-03-01,393.072784
4,1950-03-08,2873.386661,1950-03-06 22:00:00,1950-03-08,117.258295
...,...,...,...,...,...
718,2022-09-22,5556.971814,2022-09-21 16:00:00,2022-09-22,759.445786
719,2022-11-11,907.817141,2022-11-10 03:00:00,2022-11-11,34.525026
720,2022-11-30,316.819875,2022-11-29 05:00:00,2022-11-30,467.350841
721,2022-12-14,1187.444094,2022-12-13 18:00:00,2022-12-14,1589.295030


In [3]:
###Try importing the climo files from Andrew###

#df_climo_850t = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/Climos_REV/climo_850hPatemp_mean.nc')
#df_climo_pmsl = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/Climos_REV/climo_pmsl_mean.nc')
#df_climo_tcwv = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/Climos_REV/climo_tcwv_mean.nc')
#df_climo_tcwv

In [4]:
###Add 360 to the lons and change var names###

# def munge_climos(df):
    
#     '''Adds 360 to all lons and also renames variables so they match my data. '''

#     #add 360 to every longitude then wrap into [0,360)#
#     df_360 = df.assign_coords(lon=lambda ds: (ds.lon + 360))

#     #now sort the dataset along the new lon axis so that 0→360 is in order#
#     df_360 = df_360.sortby('lon')

#     #change var names#
#     df_360 = df_360.rename({'lat':'latitude', 'lon':'longitude', 'time':'valid_time'})
#     df_360 = df_360.rename({'pw':'tcwv'})
        
#     return df_360
    
# #df_clim_850t = munge_climos(df_climo_850t)
# #df_clim_msl = munge_climos(df_climo_pmsl)
# df_clim_tcwv = munge_climos(df_climo_tcwv)
# df_clim_tcwv

In [5]:
###Extract the max times so we can use them for compositing###

#Pull out the max time column#
max_times = list(df_select['max_time'].values)
dates = list(df_select['date'].values)
N = len(max_times)

max_times[0:10], print(N)


723


(['1950-01-03 09:00:00',
  '1950-01-29 06:00:00',
  '1950-02-21 16:00:00',
  '1950-02-28 07:00:00',
  '1950-03-06 22:00:00',
  '1950-03-11 01:00:00',
  '1950-03-18 23:00:00',
  '1950-03-26 17:00:00',
  '1950-04-03 22:00:00',
  '1950-04-18 12:00:00'],
 None)

In [8]:
###Example of one of the data files to see the variables###

# testing1 = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_Climos_LD/Climos_pl_LD_vars/climatology_LD_sl_1950-01-01 00:00:00.nc')
# testing1.sel(pressure_level=[250]).squeeze(dim='pressure_level', drop=True)

# testing2 = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/CF_Comp_largedomain/pressure_level_vars/pl_temps_omega/ERA5_pl_temps_omega_01_01_2011.nc')
# testing2.sel(level=[850]).squeeze(dim='level', drop=True)

##Revision Stuff##
#testing3 = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/CF_Comp_largedomain/pressure_level_vars/pl_SD_REV_QG/ERA5_pl_850temps_REV_01_01_2011.nc')
#testing3

#testing4 = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/CF_Comp_largedomain/single_level_vars/sl_LD_REV/ERA5_sl_REV_01_01_2011.nc')
#testing4

# testing5 = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/CF_Comp_largedomain/pressure_level_vars/pl_SD_REV_QG/ERA5_pl_850temps_REV_01_01_2011.nc')
# testing5

# testing6 = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/CF_Comp_largedomain/single_level_vars/sl_LD_REV/ERA5_sl_REV_01_01_2011.nc')
# testing6

testing7 = xr.open_dataset('/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/CF_Comp_largedomain/single_level_vars/Precipitable_Water_LD/ERA5_sl_01_01_2011.nc')
testing7

<xarray.Dataset>
Dimensions:    (longitude: 721, latitude: 281, time: 864)
Coordinates:
  * longitude  (longitude) float32 100.0 100.2 100.5 100.8 ... 279.5 279.8 280.0
  * latitude   (latitude) float32 80.0 79.75 79.5 79.25 ... 10.5 10.25 10.0
  * time       (time) datetime64[ns] 2010-01-01 ... 2011-12-31T23:00:00
Data variables:
    tcwv       (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-09 19:39:14 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [9]:
###Fix the lists###

dates_as_strings = [str(date).split('T')[0] for date in dates]
dates_as_strings[0:5]

['1950-01-04', '1950-01-30', '1950-02-22', '1950-03-01', '1950-03-08']

In [ ]:
###Make the compositing function###

def composite_average(data_folder, date_list, time_list, variable, lat_range, lon_range, time_lag=0, anom=False):
    '''
    Calculates the composite average 2D map for the specified variable and domain.

    Parameters:
    - data_folder: Path to the folder containing the data files.
    - date_list: List of datetime strings in the format 'YYYY-MM-DD HH:MM:SS'.
    - variable: The variable to be analyzed
    - lat_range: Tuple specifying the latitude range (min_lat, max_lat).
    - lon_range: Tuple specifying the longitude range (min_lon, max_lon).
    - time_lag: Time lag in hours to be applied (default is 0).
    '''
    
    #Small function to convert datetime string to filename format#
    def datetime_to_filename(date_str):
        
        date = datetime.strptime(date_str, '%Y-%m-%d')
        
        return f'ERA5_sl_{date.month:02d}_{date.day:02d}_{date.year}.nc'          #adjust, name of files to composite 
    
    #Read one file to get the dimensions of latitude and longitude since all will be identical#
    sample_file = os.path.join(data_folder, datetime_to_filename(date_list[5]))
    sample_data = xr.open_dataset(sample_file)
    
    lat = sample_data['latitude']
    lon = sample_data['longitude']
    
    #Find the indices corresponding to the specified latitude and longitude range#
    lat_indices = np.where((lat >= lat_range[0]) & (lat <= lat_range[1]))[0]
    lon_indices = np.where((lon >= lon_range[0]) & (lon <= lon_range[1]))[0]

    #Initialize an empty 3D array to store the data#
    composite_data = np.empty((len(date_list), len(lat_indices), len(lon_indices)))

    #Loop through each case and read the data#
    for i, (date_str, time_str) in enumerate(zip(date_list, time_list)):
        
        #Convert date string to filename#
        filename = datetime_to_filename(date_str)
        file_path = os.path.join(data_folder, filename)
        
        #Read in the data file#
        data = xr.open_dataset(file_path)
        
        #Convert the time string to a datetime object and apply time lag#
        target_time = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S') + timedelta(hours=time_lag)
        
        #Select the closest time in the dataset#
        time_idx = np.argmin(np.abs(data['valid_time'].values - np.datetime64(target_time)))
        selected_time = data['valid_time'].values[time_idx]
        
        #Extract the variable data for the selected time and within the specified lat-lon range#
        if variable == 'thetaE':
            
            thetaE = calculate_ThetaE(data)
            #print(thetaE)
            var_data = thetaE.sel(time=selected_time).isel(latitude=lat_indices, longitude=lon_indices)
            
        elif variable == 'FLF':
            
            thetaE = calculate_ThetaE(data)
            
            FLF_data = xr.apply_ufunc(
                calculate_flf, 
                thetaE, 
                input_core_dims=[['latitude', 'longitude']],  #Core dimensions over which to apply the function
                output_core_dims=[['latitude', 'longitude']], #Core dimensions on the output
                vectorize=True,  #Automatically vectorize the computation if needed
                dask='parallelized',  #Use Dask for parallel computation if thetaE is a Dask-backed array
                kwargs={'dx': dx, 'dy': dy}  #Additional keyword arguments for the function
                )
            FLF_data_n = FLF_data.where(FLF_data < 0, 0)
            
            var_data = FLF_data_n.sel(time=selected_time).isel(latitude=lat_indices, longitude=lon_indices)
            
        else:
               
            #var_data = data[variable].sel(valid_time=selected_time).isel(latitude=lat_indices, longitude=lon_indices).sel(pressure_level=[500]).squeeze(dim='pressure_level', drop=True) 
            var_data = data[variable].sel(valid_time=selected_time).isel(latitude=lat_indices, longitude=lon_indices)                      #change for sl or pl
            
            global var_data_lats, var_data_lons
            var_data_lats = var_data['latitude']
            var_data_lons = var_data['longitude']
            
            if anom == True:
                
                #Turn the selected_time into a 0-based day-of-year index#
                ts = pd.to_datetime(str(selected_time))
                
                if ts.is_leap_year:
                    
                    if ts.dayofyear == 60:
                        doy = 58
                        
                    elif ts.dayofyear > 60:
                        doy = ts.dayofyear - 2
                    
                    else:
                        doy = ts.dayofyear - 1
                else:
                    #Ordinary year#
                    doy = ts.dayofyear - 1

                #Pick the right climatology variable#
                if variable == 't':
                    cl = df_clim_850t['t']
                    
                elif variable == 'msl':
                    cl = df_clim_msl['msl']
                    
                elif variable == 'tcwv':
                    cl = df_clim_tcwv['tcwv']
                    
                else:
                    raise KeyError(f"No preloaded climo for variable {variable!r}")

                #extract the 2D slice at the correct day and subset lat/lon#
                clim_slice = (cl.isel(valid_time=doy))
                
                clim_on_data = clim_slice.interp(
                latitude = var_data.latitude,
                longitude= var_data.longitude
                )
                
                #Calculate anomalies#
                var_data = var_data.values - clim_on_data
                
            else:
                var_data = var_data.copy()
        
        #Store the data in the array#
        composite_data[i, :, :] = var_data

    #Calculate the composite average along the first axis (CF cases)#
    composite_avg = np.mean(composite_data, axis=0)            #mean or std
    
    print("Done Compositing!")

    return composite_avg, composite_data


#Obtaining composite stuff#
data_folder = '/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/CF_Comp_largedomain/pressure_level_vars/pl_QG_attempt2/'   #may have to edit
date_list = dates_as_strings.copy()
time_list = max_times.copy()
variable = 'tcwv'             #any variable in xr (thetaE, FLF too)
lat_range = (34, 44)          #Latitude range (min_lat, max_lat): state of CO with 1 deg buffer #35,43  / 10,70
lon_range = (-112, -99)        #Longitude range (min_lon, max_lon): state of CO with 1 deg buffer  #-110,-100  / 220,290
time_lag = 0                  #Can either be 0 or negative numbers

composite_avg, composite_data = composite_average(data_folder, date_list, time_list, variable, lat_range, lon_range, time_lag, anom=True)

In [ ]:
composite_data.shape

In [ ]:
###Save this variables###

#tcwv REV#
#t500_HP_l0 = composite_avg.copy()
#t500_LP_l0 = composite_avg.copy()
t500_All_l0 = composite_avg.copy()

#Specify the dimensions to save#
lat = var_data_lats 
lon = var_data_lons

# Create DataArrays with explicit dimensions for each variable
#t500_HP_l0 = xr.DataArray(t500_HP_l0, dims=["latitude", "longitude"], coords={"latitude": lat, "longitude": lon})
#t500_LP_l0 = xr.DataArray(t500_LP_l0, dims=["latitude", "longitude"], coords={"latitude": lat, "longitude": lon})
t500_All_l0 = xr.DataArray(t500_All_l0, dims=["latitude", "longitude"], coords={"latitude": lat, "longitude": lon})

# Combine them into a Dataset
# ds_t850 = xr.Dataset({
#     'w700_HP_l0': w700_HP_l0,
#     'w700_LP_l0': w700_LP_l0,
#     'w700_All_l0': w700_All_l0,
# })

# Save the dataset to a NetCDF file
t500_All_l0.to_netcdf('t500_All_l0.nc')


In [ ]:
###Plot the composite map for Colorado Region###

#Specify a variable and units to plot#
var_of_interest = '2-m Temperature'
unit_of_interest = 'C'

#Set up the projection that will be used#
mapcrs = ccrs.LambertConformal(central_longitude=-55, central_latitude=45, standard_parallels=(33, 45)) 
mapcrs = ccrs.PlateCarree()

#Set up the projection of the data; if lat/lon then PlateCarree is what you want#
datacrs = ccrs.PlateCarree()

#Start the figure and create plot axes with proper projection#
fig = plt.figure(1, figsize=(12, 12)) 
ax = plt.subplot(111, projection=mapcrs) 
ax.set_extent([-110, -100, 35, 43], ccrs.PlateCarree())  #-110, -100, 35, 43 

#Add geopolitical boundaries for map reference#
ax.add_feature(cfeature.LAND, facecolor="white") 
countries = NaturalEarthFeature(category="cultural", scale="110m", facecolor="none", name="admin_0_boundary_lines_land") 
ax.add_feature(countries, linewidth=0.5, edgecolor="black") 
ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.5) 
ax.coastlines('50m', linewidth=0.8)

#Add the fcontours#
#temp_levels = np.arange(264, 289, 0.5)
#temp_levels_anom = np.arange(-8, 8.5, 0.5)
#temp_levels_std = np.arange(4, 7.1, 0.1)
#dew_levels = np.arange(260, 275, 0.5)
#dew_levels_anom = np.arange(-3.5, 3.7, 0.2)
#dew_levels_std = np.arange(3.5, 6.6, 0.1)
#mslp_levels = np.arange(1004, 1025, 1)
#u_v_levels = np.arange(-8, 4.5, 0.5)
#thetaE_levels = np.arange(288, 313, 1)
#FLF_levels = np.arange(-2.5, 0.5, 0.1)
#pw_levels = np.arange(0, 14.5, 0.5)
#omega_levels = np.arange(-1, 1.05, 0.05)
#precip_levels = np.arange(0, 0.00014, 0.00001)
#ptype_levels = np.arange(0, 9, 1)
temp_comp_cf = plt.contourf(var_data_lons, var_data_lats, composite_avg, levels=None, cmap="bwr", alpha=1, transform=ccrs.PlateCarree(), label="2-m Temp")

#Colorbar and contour labels#
cb = fig.colorbar(temp_comp_cf, orientation='horizontal', pad=0.08, extendrect=True, aspect=20, shrink=1)
cb.set_label(f'{var_of_interest} [{unit_of_interest}]', size='x-large') #FLF [K/m^3 * 10^14]

#Fix the lats and lon values#
ax.set_xticks(range(-111, -99, 1), crs=ccrs.PlateCarree())
ax.set_yticks(range(35, 44, 1), crs=ccrs.PlateCarree())
x_ticks = [-110,-108,-106,-104,-102,-100]
x_tick_labels = ['-110°','-108°','-106°','-104°','-102°','-100°']
y_ticks = [35,37,39,41,43]
y_tick_labels = ['35°','37°','39°','41°','43°']
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)
ax.tick_params(axis='both', which='major', labelsize=14)

#Plot the longitude and latitude#
ax.set_xticks(range(-110, -99, 1), crs=ccrs.PlateCarree())
ax.set_yticks(range(35, 44, 1), crs=ccrs.PlateCarree())
ax.set_xlabel('Longitude', fontsize=14)
ax.set_ylabel('Latitude', fontsize=14)
plt.title(f'Composited {var_of_interest} [{unit_of_interest}] for timelag={time_lag}', fontsize=16) 

plt.show()

In [ ]:
###Plot the composite map for Full Domain###

#Set up the projection that will be used#
mapcrs = ccrs.LambertConformal(central_longitude=-55, central_latitude=45, standard_parallels=(33, 45)) 
mapcrs = ccrs.PlateCarree(central_longitude=180)

#Set up the projection of the data; if lat/lon then PlateCarree is what you want#
datacrs = ccrs.PlateCarree(central_longitude=180)

#Start the figure and create plot axes with proper projection#
fig = plt.figure(1, figsize=(14, 14)) 
ax = plt.subplot(111, projection=mapcrs) 
ax.set_extent([220, 290, 10, 70], ccrs.PlateCarree())  #-110, -100, 35, 43 
gl = ax.gridlines(crs=ccrs.PlateCarree(),linestyle="--", linewidth=1, color='grey', alpha=1)

#Add geopolitical boundaries for map reference#
ax.add_feature(cfeature.LAND, facecolor="white") 
countries = NaturalEarthFeature(category="cultural", scale="110m", facecolor="none", name="admin_0_boundary_lines_land") 
ax.add_feature(countries, linewidth=0.5, edgecolor="black") 
ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.5) 
ax.coastlines('50m', linewidth=0.8)

#Add the fcontours#
#pw_levels = np.arange(0, 12.5, 0.1)
#msl_levels = np.arange(1002, 1030, 2)
#geop_levels = np.arange(940, 1080, 8)
#omega_levels = np.arange(-1.2, 1.3, 0.1)
#temp_levels = np.arange(-24, 26, 2)
#wind_levels_u = np.arange(-8, 60, 4)
#wind_levels_v = np.arange(-25, 25, 3)
#wind_speed_levels = np.arange(0, 60, 4)
#OLR_levels = np.arange(1.1, 0.52, 0.02)
temp_comp_cfld = plt.contourf(var_data_lons, var_data_lats, composite_avg, levels=None, cmap="bwr", alpha=1, transform=ccrs.PlateCarree(), label="2-m Temp")

#Add labels to the contours#
#plt.clabel(temp_comp_cfld, inline=True, fontsize=8, fmt='%1.1f')

#Colorbar and contour labels#
cbld = fig.colorbar(temp_comp_cfld, orientation='horizontal', pad=0.08, extendrect=True, aspect=20, shrink=1)
cbld.set_label(f'{var_of_interest} [{unit_of_interest}]', size='x-large') #FLF [K/m^3 * 10^14]

#Fix the lats and lon values#
#ax.set_xticks(range(-180, -70, 10), crs=ccrs.PlateCarree())
ax.set_yticks(range(10, 70, 10), crs=ccrs.PlateCarree())
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticks(range(220, 290, 10), crs=ccrs.PlateCarree())

#Set labels for plot#
ax.set_xlabel('Longitude', fontsize=14)
ax.set_ylabel('Latitude', fontsize=14)
plt.title(f'Composited {var_of_interest} [{unit_of_interest}] for timelag={time_lag} hours', fontsize=16) 

plt.show()